# custom implementation of code

# Loading model

In [1]:
import torch
from boxmot import DeepOCSORT
from pathlib import Path

model = torch.hub.load('C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\yolov7-main\\yolov7-main', 'custom', "C:\\Users\\jithi\\Downloads\\yolov7.pt",force_reload=True, source='local',trust_repo=True)
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'),
    device='cuda:0',
    fp16=True,
)

c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]   

Adding autoShape... 


Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: c:\Users\jithi\OneDrive\Desktop\VsCode\capstone\notebooks\osnet_x0_25_msmt17.pt
100%|██████████| 3.06M/3.06M [00:00<00:00, 9.04MB/s]
2023-11-22 16:19:34.605 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-11-22 16:19:34.608 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


### format of predictions

In [3]:
results = model("C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\sample.jpeg")
df = results.pandas().xyxy[0]
df

c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


,xmin,ymin,xmax,ymax,confidence,class,name
0,0.147656,24.042187,64.706245,148.500000,0.946289,0,person
1,278.512482,16.462500,335.737488,147.056244,0.923828,0,person
2,225.749985,39.923435,281.137482,149.550003,0.919922,0,person
3,112.218742,27.192186,189.656235,148.631241,0.916992,0,person
4,58.504684,34.771873,121.143745,149.550003,0.913086,0,person
5,174.956238,37.035934,229.818741,149.024994,0.886719,0,person


### Detections On Image

In [4]:
import cv2
import numpy as np
from pathlib import Path
from boxmot import DeepOCSORT

# Load the image for object detection
image_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\sample.jpeg"  # Replace with the path to your image
frame = cv2.imread(image_path)

# Load DeepOCSORT tracker
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'),  # Replace with your ReID model weights
    device='cuda:0',
    fp16=True,
)

color = (0, 0, 255)  # BGR color for bounding box and text
thickness = 2  # Thickness of bounding box lines
fontscale = 0.5  # Font scale for text

# Perform object detection using your model and get bounding boxes and class names
# Replace this part with your actual object detection code
results = model(frame)  # Modify this line to match your object detection model
df = results.pandas().xyxy[0]
detections = []
for _, row in df.iterrows():
    x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
    class_name = row['class']
    detections.append([x1, y1, x2, y2, 1.0, class_name])  # Confidence set to 1.0

# Update the DeepOCSORT tracker with detections
tracks = tracker.update(np.array(detections), frame)

xyxys = tracks[:, 0:4].astype('int')
ids = tracks[:, 4].astype('int')
confs = tracks[:, 5]
clss = tracks[:, 6].astype('int')
inds = tracks[:, 7].astype('int')

if tracks.shape[0] != 0:
    for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
        frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
        text = f' class: {cls}'
        frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
        print(detections)

# Display the image with Object IDs and Classes
cv2.imshow('Image with Object IDs and Classes', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


2023-11-22 16:20:16.616 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-11-22 16:20:16.618 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


[[278, 15, 335, 146, 1.0, 0], [0, 24, 64, 149, 1.0, 0], [112, 26, 187, 148, 1.0, 0], [227, 39, 281, 149, 1.0, 0], [58, 34, 122, 149, 1.0, 0], [174, 36, 229, 149, 1.0, 0]]
[[278, 15, 335, 146, 1.0, 0], [0, 24, 64, 149, 1.0, 0], [112, 26, 187, 148, 1.0, 0], [227, 39, 281, 149, 1.0, 0], [58, 34, 122, 149, 1.0, 0], [174, 36, 229, 149, 1.0, 0]]
[[278, 15, 335, 146, 1.0, 0], [0, 24, 64, 149, 1.0, 0], [112, 26, 187, 148, 1.0, 0], [227, 39, 281, 149, 1.0, 0], [58, 34, 122, 149, 1.0, 0], [174, 36, 229, 149, 1.0, 0]]
[[278, 15, 335, 146, 1.0, 0], [0, 24, 64, 149, 1.0, 0], [112, 26, 187, 148, 1.0, 0], [227, 39, 281, 149, 1.0, 0], [58, 34, 122, 149, 1.0, 0], [174, 36, 229, 149, 1.0, 0]]
[[278, 15, 335, 146, 1.0, 0], [0, 24, 64, 149, 1.0, 0], [112, 26, 187, 148, 1.0, 0], [227, 39, 281, 149, 1.0, 0], [58, 34, 122, 149, 1.0, 0], [174, 36, 229, 149, 1.0, 0]]
[[278, 15, 335, 146, 1.0, 0], [0, 24, 64, 149, 1.0, 0], [112, 26, 187, 148, 1.0, 0], [227, 39, 281, 149, 1.0, 0], [58, 34, 122, 149, 1.0, 0], [17

In [10]:
import cv2
import pandas as pd

# Initialize video capture
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference using your model (replace this with your actual model)
    results = model(frame)

    # Convert results to pandas DataFrame
    df = results.pandas().xyxy[0]

    # Loop through each bounding box in the DataFrame
    for index, row in df.iterrows():
        # Extract bounding box coordinates
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        
        # Extract class name (replace 'class_name' with the actual column name from your DataFrame)
        class_name = row['name']

        # Draw rectangle on the frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle with thickness 2

        # Draw class name
        cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Display the frame with bounding boxes
    cv2.imshow('Video with Bounding Boxes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()


In [11]:
import cv2
import numpy as np
from pathlib import Path
from boxmot import DeepOCSORT

# Initialize video capture for object detection
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)

# Load DeepOCSORT tracker
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'),  # Replace with your ReID model weights
    device='cuda:0',
    fp16=True,
)

color = (0, 0, 0)  # BGR color for bounding box and text
thickness = 2  # Thickness of bounding box lines
fontscale = 0.5  # Font scale for text

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)  
    df = results.pandas().xyxy[0]
    detections = []
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        class_name = row['class']
        detections.append([x1, y1, x2, y2, 1.0, class_name])  # Confidence set to 1.0

    # Update the DeepOCSORT tracker with detections
    print(detections)
    tracks = tracker.update(np.array(detections), frame)

    xyxys = tracks[:, 0:4].astype('int')
    ids = tracks[:, 4].astype('int')
    confs = tracks[:, 5]
    clss = tracks[:, 6].astype('int')
    inds = tracks[:, 7].astype('int')

    if tracks.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
            text = f'id: {id}, class: {cls}, conf: {conf}'
            frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)

    cv2.imshow('Video with Object IDs and Classes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()

2023-10-06 15:12:13.919 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-10-06 15:12:13.921 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


[[635, 261, 876, 761, 1.0, 0], [459, 252, 663, 764, 1.0, 0], [409, 175, 454, 300, 1.0, 0], [1268, 202, 1329, 367, 1.0, 0], [358, 659, 535, 776, 1.0, 16], [516, 188, 571, 258, 1.0, 0], [799, 17, 841, 58, 1.0, 2], [966, 56, 981, 101, 1.0, 0], [947, 59, 962, 100, 1.0, 0], [644, 201, 764, 319, 1.0, 2], [720, 120, 789, 209, 1.0, 2], [745, 94, 811, 163, 1.0, 2], [876, 567, 916, 594, 1.0, 14], [784, 95, 811, 161, 1.0, 2], [744, 54, 801, 92, 1.0, 2]]
[[635, 261, 876, 761, 1.0, 0], [459, 252, 664, 764, 1.0, 0], [409, 175, 454, 299, 1.0, 0], [1268, 202, 1329, 367, 1.0, 0], [359, 659, 535, 776, 1.0, 16], [516, 188, 571, 258, 1.0, 0], [799, 17, 841, 58, 1.0, 2], [966, 56, 981, 101, 1.0, 0], [947, 59, 962, 100, 1.0, 0], [644, 200, 764, 319, 1.0, 2], [720, 120, 789, 209, 1.0, 2], [745, 94, 811, 163, 1.0, 2], [876, 567, 916, 595, 1.0, 14], [784, 95, 811, 161, 1.0, 2], [744, 54, 801, 92, 1.0, 2]]
[[635, 265, 886, 759, 1.0, 0], [408, 177, 452, 301, 1.0, 0], [471, 251, 669, 760, 1.0, 0], [1267, 206, 132

In [12]:
import cv2
import numpy as np
from pathlib import Path
from boxmot import DeepOCSORT

# Initialize video capture for object detection
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)

# # Load DeepOCSORT tracker
# tracker = DeepOCSORT(
#     model_weights=Path('osnet_x0_25_msmt17.pt'),
#     device='cuda:0',
#     fp16=True,
# )

# Define colors for each class
class_colors = {
    0: (0, 0, 255),
    1: (0, 255, 0),
    16: (255, 0, 0),
    # Add more classes and their colors
}

thickness = 2
fontscale = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    df = results.pandas().xyxy[0]
    detections = []
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        class_name = row['class']
        detections.append([x1, y1, x2, y2, 1.0, class_name])

    tracks = tracker.update(np.array(detections), frame)

    xyxys = tracks[:, 0:4].astype('int')
    ids = tracks[:, 4].astype('int')
    confs = tracks[:, 5]
    clss = tracks[:, 6].astype('int')
    inds = tracks[:, 7].astype('int')

    if tracks.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            color = class_colors.get(cls, (0, 0, 0))  # Default color is black
            frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
            text = f'id: {id}, class: {cls}, conf: {conf}'
            frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)

    cv2.imshow('Video with Object IDs and Classes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# gradi interface for starting

In [1]:
import gradio as gr
import json

# Initialize checkbox states
checkbox_states = {"person": False, "cat": False, "car": False, "truck": False,"bicycle":False,"motorcycle":False,"dog":False}

def update_checkboxes(person, cat, car, truck,bicycle,motorcycle,dog):
    checkbox_states["person"] = person
    checkbox_states["cat"] = cat
    checkbox_states["car"] = car
    checkbox_states["truck"] = truck
    checkbox_states["bicycle"] = bicycle
    checkbox_states["motorcycle"] = motorcycle
    checkbox_states["dog"] = dog
    
    
    # Write to JSON file
    with open("checkbox_states.json", "w") as f:
        json.dump(checkbox_states, f)

    return f"Updated and saved checkbox states: {checkbox_states}"

demo = gr.Interface(
    update_checkboxes,
    [
        gr.Checkbox(initial_value=checkbox_states["person"], label="person"),
        gr.Checkbox(initial_value=checkbox_states["cat"], label="cat"),
        gr.Checkbox(initial_value=checkbox_states["car"], label="car"),
        gr.Checkbox(initial_value=checkbox_states["truck"], label="truck"),
        gr.Checkbox(initial_value=checkbox_states["bicycle"], label="bicycle"),
        gr.Checkbox(initial_value=checkbox_states["motorcycle"], label="motorcycle"),
        gr.Checkbox(initial_value=checkbox_states["dog"], label="dog")
    ],
    "text"
)

if __name__ == "__main__":
    demo.launch()

c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\jithi\AppData\Local\Temp\ipykernel_25300\4073465248.py:26: GradioUnusedKwargWarning: You have unused kwarg parameters in Checkbox, please remove them: {'initial_value': False}
  gr.Checkbox(initial_value=checkbox_states["person"], label="person"),
C:\Users\jithi\AppData\Local\Temp\ipykernel_25300\4073465248.py:27: GradioUnusedKwargWarning: You have unused kwarg parameters in Checkbox, please remove them: {'initial_value': False}
  gr.Checkbox(initial_value=checkbox_states["cat"], label="cat"),
C:\Users\jithi\AppData\Local\Temp\ipykernel_25300\4073465248.py:28: GradioUnusedKwargWarning: You have unused kwarg parameters in Checkbox, please remove them: {'initial_value': False}
  gr.Checkbox(initial_value=checkbox_states[

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# logging part

In [3]:
import csv
from datetime import datetime
# Initialize CSV file
csv_file = open('logs.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Time', 'ID', 'Class'])


15

# filtering out classes

In [12]:
import cv2
import pandas as pd
import json

# Initialize video capture
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)

# Load JSON file that contains checkbox states
json_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\checkbox_states.json"
with open(json_path, 'r') as f:
    checkbox_states = json.load(f)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)  
    df = results.pandas().xyxy[0]  
    json_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\checkbox_states.json"
    with open(json_path, 'r') as f:
        checkbox_states = json.load(f)
    # Loop through each bounding box in the DataFrame
    for index, row in df.iterrows():
        # Extract bounding box coordinates
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

        # Extract class name
        class_name = row['name']

        # Check if this class should have a bounding box drawn, according to the JSON file
        if checkbox_states.get(class_name, False):
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Display the frame with bounding boxes
    cv2.imshow('Video with Bounding Boxes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()


# logic for tracking 

In [27]:
import cv2
import numpy as np
from datetime import datetime
# Initialize video capture for object detection
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)
import csv
# Load DeepOCSORT tracker


# Define colors for each class
class_colors = {
    0: (0, 0, 255),
    2: (0, 255, 0),
    16: (255, 0, 0),
    # Add more classes and their colors
}

thickness = 2
fontscale = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    df = results.pandas().xyxy[0]
    detections = []

        # Check if this class should have a bounding box drawn, according to the JSON file
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        class_name = row['class']
        detections.append([x1, y1, x2, y2, 1.0, class_name])

    tracks = tracker.update(np.array(detections), frame)

    xyxys = tracks[:, 0:4].astype('int')
    ids = tracks[:, 4].astype('int')
    confs = tracks[:, 5]
    clss = tracks[:, 6].astype('int')
    inds = tracks[:, 7].astype('int')

    if tracks.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            color = class_colors.get(cls, (0, 0, 0))  # Default color is black
            frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
            text = f'id: {id}, class: {cls}, conf: {conf}'
            frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            csv_writer.writerow([current_time, id, cls])    

    cv2.imshow('Video with Object IDs and Classes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# cumulativev

In [1]:
import cv2
import numpy as np
import json
import csv
from datetime import datetime
# Initialize CSV file
csv_file = open('logs.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Time', 'ID', 'Class'])


NameError: name 'model' is not defined

In [ ]:
id_to_name = {
    0: 'person',
    2: 'car',
    14: 'bird',
    # 16:'none',
    # 7:'bird',
    # 58:'pot',
    # 10:'none'
    
}

# Initialize video capture for object detection
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)
json_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\checkbox_states.json"
    
# Load DeepOCSORT tracker


# Define colors for each class
class_colors = {
    0: (0, 0, 255),
    1: (0, 255, 0),
    14: (255, 0, 0),
    # Add more classes and their colors
}

thickness = 2
fontscale = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    with open(json_path, 'r') as f:
        checkbox_states = json.load(f)
   
    results = model(frame)
    df = results.pandas().xyxy[0]
    detections = []
    for _, row in df.iterrows():
        
        class_name = row['class']
        class_str=id_to_name.get(class_name)
        if checkbox_states.get(str(class_str), False):  # Only add if class_name exists with True in checkbox_states
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            detections.append([x1, y1, x2, y2, 1.0, class_name])
            
               
              
        
    print(detections)
    tracks = tracker.update(np.array(detections), frame)
    # print(tracks)
    xyxys = tracks[:, 0:4].astype('int')
    ids = tracks[:, 4].astype('int')
    confs = tracks[:, 5]
    clss = tracks[:, 6].astype('int')
    inds = tracks[:, 7].astype('int')

    if tracks.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            color = class_colors.get(cls, (0, 0, 0))  # Default color is black
            frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
            text = f'id: {id}, class: {cls}, conf: {conf}'
            frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            csv_writer.writerow([current_time, id, cls])    

    cv2.imshow('Video with Object IDs and Classes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# logic for logging

In [4]:
csv_file = open('logs.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Time', 'ID', 'Class'])


15

In [3]:
import cv2
import numpy as np
from datetime import datetime
# Initialize video capture for object detection
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)
import csv
csv_file = open('logs.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Time', 'ID', 'Class'])

# Define colors for each class
class_colors = {
    0: (0, 0, 255),
    1: (0, 255, 0),
    16: (255, 0, 0),
    # Add more classes and their colors
}

thickness = 2
fontscale = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    df = results.pandas().xyxy[0]
    detections = []

        # Check if this class should have a bounding box drawn, according to the JSON file
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        class_name = row['class']
        detections.append([x1, y1, x2, y2, 1.0, class_name])

    tracks = tracker.update(np.array(detections), frame)

    xyxys = tracks[:, 0:4].astype('int')
    ids = tracks[:, 4].astype('int')
    confs = tracks[:, 5]
    clss = tracks[:, 6].astype('int')
    inds = tracks[:, 7].astype('int')

    if tracks.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            color = class_colors.get(cls, (0, 0, 0))  # Default color is black
            frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
            text = f'id: {id}, class: {cls}, conf: {conf}'
            frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            csv_writer.writerow([current_time, id, cls])    

    cv2.imshow('Video with Object IDs and Classes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
import cv2
import numpy as np
from pathlib import Path
from boxmot import DeepOCSORT
import csv
import cv2
import torch
import numpy as np
from pathlib import Path
from datetime import datetime
from boxmot import DeepOCSORT
# Initialize video capture for object detection
video_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
cap = cv2.VideoCapture(video_path)

# Load DeepOCSORT tracker


# Define colors for each class
class_colors = {
    0: (0, 0, 255),
    1: (0, 255, 0),
    16: (255, 0, 0),
    # Add more classes and their colors
}

thickness = 2
fontscale = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    df = results.pandas().xyxy[0]
    detections = []
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        class_name = row['class']
        detections.append([x1, y1, x2, y2, 1.0, class_name])

    tracks = tracker.update(np.array(detections), frame)

    xyxys = tracks[:, 0:4].astype('int')
    ids = tracks[:, 4].astype('int')
    confs = tracks[:, 5]
    clss = tracks[:, 6].astype('int')
    inds = tracks[:, 7].astype('int')

    if tracks.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            color = class_colors.get(cls, (0, 0, 0))  # Default color is black
            frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
            text = f'id: {id}, class: {cls}, conf: {conf}'
            frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            csv_writer.writerow([current_time, id, cls])    

    cv2.imshow('Video with Object IDs and Classes', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
